In [ ]:
%cd ..

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
models = [
    "dae_0.04314",
    "drunet_0.0-0.2",
    "dcnn_0.05",
    "drcnn_0.05",
    "dunet_0.05",
    "dunet+_0.05",
    "dunet+_0.0-0.2",
    "drunet+_0.0-0.2",
    "drugan+-lambda-zero_0.0-0.2",
    "drugan+_0.0-0.2",
]
metrics = ["psnr", "fsim"]
noise_levels = ["0.05", "0.10", "0.20"]
datasets = ["bsds500/gauss"]

In [ ]:
# Read the evaluation
df_all = pd.read_csv('results/denoising_evaluation.csv')

# Read the model name mapping
with open('models/model_name_mapping.json') as f:
    model_mapping = json.load(f)


In [ ]:
df = df_all.copy()

# Only take rows of the configured models
df = df[df['Model'].isin(models)]

# Split the dataset name
df[['Dataset', 'Dataset_Variant']] = df['Dataset'].str.split("_", expand=True)

# Filter datasets
df = df[df['Dataset'].isin(datasets)]

# Mean over the images
df = df.groupby(['Model', 'Dataset_Variant']).mean()
df = df.reset_index()
df = df.drop(columns=['Image_Id'])

# Combine all metrics in one column
df = df.melt(id_vars=['Model', 'Dataset_Variant',], var_name='metric', value_name='value')

# Filter: Only configured metrics, only configured noise levels
df = df[df['metric'].isin(metrics)]
df = df[df['Dataset_Variant'].isin(noise_levels)]

# Set values to NaN where the model is not trained for the noise
df['value'][~df['Model'].str.endswith('0.2') & (df['Dataset_Variant'] != '0.05')] = np.nan

# Sort
df = df.sort_values('metric', kind='mergesort', key=lambda x: [0 if v == "PSNR" else 1 for v in x])
df = df.sort_values('Dataset_Variant', kind='mergesort')

# Rename metrics
df['metric'] = df['metric'].map({'fsim': 'FSIM', 'psnr': 'PSNR', 'ssim': 'SSIM'})

# Set the index
df = df.set_index(['Model', 'Dataset_Variant', 'metric'])

# Unstack noise_stddev
df = df.unstack(['Dataset_Variant', 'metric'])

# Organize the column naming
df.columns = df.columns.droplevel(0)
df.columns = df.columns.rename(['Additive noise \( \sigma_n \)', ''])

df = df.sort_values('Model', key=lambda x: [models.index(v) for v in x], kind='mergesort')
df.index = df.index.map(model_mapping)

df

In [ ]:
def float_format(v):
    if v < 1.0:
        return f"{v:.4f}" #[1:] <-- Do not show leading 0
    else:
        return f"{v:.2f}"

column_format = 'l' + 'c' * len(df.columns)

with pd.option_context("max_colwidth", 1000):
    latex = df.to_latex(float_format=float_format,
                        escape=False,
                        na_rep='---',
                        column_format=column_format,
                        multicolumn_format='c')
print(latex)